In [1]:
import torch
import onnx
import onnxruntime
import numpy as np
import cv2
from torchvision.transforms import transforms

In [3]:
net = torch.jit.load("/home/mohammad/Documents/gender_detection/Abolfazl_gender_detection.pt").to('cpu')
net.eval()

RecursiveScriptModule(
  original_name=AbolfazNework
  (pretrained_model): RecursiveScriptModule(
    original_name=ResNet
    (conv1): RecursiveScriptModule(original_name=Conv2d)
    (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
    (relu): RecursiveScriptModule(original_name=ReLU)
    (maxpool): RecursiveScriptModule(original_name=MaxPool2d)
    (layer1): RecursiveScriptModule(
      original_name=Sequential
      (0): RecursiveScriptModule(
        original_name=BasicBlock
        (conv1): RecursiveScriptModule(original_name=Conv2d)
        (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
        (relu): RecursiveScriptModule(original_name=ReLU)
        (conv2): RecursiveScriptModule(original_name=Conv2d)
        (bn2): RecursiveScriptModule(original_name=BatchNorm2d)
      )
      (1): RecursiveScriptModule(
        original_name=BasicBlock
        (conv1): RecursiveScriptModule(original_name=Conv2d)
        (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
 

In [6]:
# Input to the model
x = torch.randn(1, 3, 224, 224, requires_grad=True).to('cpu')
torch_out = net(x)

# Export the model
torch.onnx.export(net,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "/home/mohammad/Documents/gender_detection/ff.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'}, 'output' : {0 : 'batch_size'}})

/home/mohammad/Documents/hair_segmentation(trained_verstion)/env3.7/lib/python3.7/site-packages/torch/onnx/utils.py:823: UserWarning: no signature found for <torch.ScriptMethod object at 0x7f9c68634bf8>, skipping _decide_input_format
  warnings.warn(f"{e}, skipping _decide_input_format")


In [7]:
onnx_model = onnx.load("/home/mohammad/Documents/gender_detection/ff.onnx")
onnx.checker.check_model(onnx_model)

In [8]:
ort_session = onnxruntime.InferenceSession("/home/mohammad/Documents/gender_detection/ff.onnx")


In [9]:
ort_session = onnxruntime.InferenceSession("/home/mohammad/Documents/gender_detection/ff.onnx")

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
ort_outs = ort_session.run(None, ort_inputs)

# compare ONNX Runtime and PyTorch results
np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)

print("Exported model has been tested with ONNXRuntime, and the result looks good!")

Exported model has been tested with ONNXRuntime, and the result looks good!


In [35]:
image = cv2.imread("/home/mohammad/Documents/GenderClassifiey/all_tracked_faces/121/Last_frame_number_2102_IQA_score_0.080791_Camera_Number_0_area_conf_0.484589_.jpg")
tr = transforms.Compose([transforms.ToTensor(), transforms.Resize((224, 224))])
image = tr(image).reshape((1, 3, 224, 224))

In [36]:
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(image)}
ort_outs = ort_session.run(None, ort_inputs)
img_out_y = ort_outs[0]

In [37]:
print(img_out_y)

[[0.7094111  0.29058892]]
